<a href="https://colab.research.google.com/github/FubiniBeanie/BlueMicro/blob/main/Excel_Automation_Intern_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Import Excel Sheet for Automation

This block will ask you to import a UPC unverse excel sheet. It will save an updated one in the file folder on the left side of the Google Colab Screen.

Open the block with the **drop down ">"** and
start pressing the Play button on the left to run the code. You will have to run just one block and then will be promted at the **bottom** of the block to import your excel file.

In [5]:
## Import dataframe:


import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.worksheet.worksheet import Worksheet
from google.colab import files
import numpy as np


# 2. Import Data into Dataframe

# Function to fetch data from the worksheet upload
def fetch_supplier_data():
  # Upload the Excel file
  uploaded = files.upload()

  # Get the name of the first uploaded file
  excel_file_name = list(uploaded.keys())[0]

  # Remove the .xlsx extension
  excel_file_name = excel_file_name.rstrip('.xlsx')

  # Load the uploaded Excel file directly into a pandas DataFrame
  df = pd.read_excel(excel_file_name + '.xlsx')
  return df, excel_file_name

  # Display the first few rows of the DataFrame
  #print(df.head())
  # Confirm that df is a pandas DataFrame
  #print("Type of df:", type(df))

# 3. Create Function for filtering

# Function to determine the action based on row values
def determine_action(row):
    smarterx_status = row['SmarterX Active Status']
    retailer_inv_status = row['Retailer Inventory Status']
    in_spins = row['In SPINS']

    # Rule 1: If both 'SmarterX Active Status' and 'Retailer Inventory Status' are 'Active'
    if smarterx_status == 'Active' and retailer_inv_status == 'Active':
        return 'Renew'

    # Rule 2: If 'SmarterX Active Status' is 'Active' and 'Retailer Inventory Status' is 'Inactive'
    elif smarterx_status == 'Active' and retailer_inv_status == 'Inactive':
        return 'Renew' if in_spins == 'Yes' else 'Move to Inactive in SmarterX'

    # Rule 3: If 'SmarterX Active Status' is 'Inactive' or null, and 'Retailer Inventory Status' is 'Active'
    elif (pd.isnull(smarterx_status) or smarterx_status == 'Inactive') and retailer_inv_status == 'Active':
        return 'Register and add to Renewal Order Form'

    # Rule 4: If 'SmarterX Active Status' is 'Inactive' or null, and 'Retailer Inventory Status' is 'Inactive'
    elif (pd.isnull(smarterx_status) or smarterx_status == 'Inactive') and retailer_inv_status == 'Inactive':
        return 'Register and add to Renewal Order Form' if in_spins == 'Yes' else 'None'

    return np.nan  # In case the row doesn't match any condition

# 5. Add colors

def apply_color_mapping(worksheet, df):
    red_fill = PatternFill(start_color="FFCCCC", end_color="FFCCCC", fill_type="solid")
    green_fill = PatternFill(start_color="CCFFCC", end_color="CCFFCC", fill_type="solid")
    yellow_fill = PatternFill(start_color="FFFFCC", end_color="FFFFCC", fill_type="solid")

    # Map actions to their corresponding fill colors
    action_to_fill = {
        'Move to Inactive in SmarterX': red_fill,
        'Renew': green_fill,
        'Register and add to Renewal Order Form': yellow_fill,
        'None': PatternFill(start_color="00FFFF", end_color="00FFFF", fill_type="solid")  # Assuming 'None' is a possible action
    }

    excel_row_start = 2
    for idx, row in df.iterrows():
        action = row['Action']
        # Convert NaN (pandas' notion of None/missing) to 'None'
        if pd.isna(action):
            action = 'None'
        fill = action_to_fill.get(action)
        if fill:
            excel_row = excel_row_start + idx
            for col_idx in range(1, len(row) + 1):
                worksheet.cell(row=excel_row, column=col_idx).fill = fill

# Main function to orchestrate the flow
def main():
    df, excel_file_name = fetch_supplier_data()
    df['Action'] = df.apply(determine_action, axis=1)



    df = df[['Action'] + [col for col in df.columns if col != 'Action']]

    #Sorts Action Values
    df.sort_values(by='Action', inplace=True)
    df.reset_index(drop=True, inplace=True)  # Reset index here

    output_xlsx_path = f"{excel_file_name}_output.xlsx"

    excel_writer = pd.ExcelWriter(output_xlsx_path, engine='openpyxl')
    df.to_excel(excel_writer, index=False, sheet_name='Data')

    worksheet = excel_writer.sheets['Data']
    apply_color_mapping(worksheet, df)

    workbook = excel_writer.book

    excel_writer.save()
    print(f"Styled DataFrame saved to XLSX file '{excel_file_name}' with an 'Data' sheet.")

if __name__ == "__main__":
    main()


IndexError: list index out of range

Once you upload your excel file you should be able to **open the folder** on the left side of the screen an be able to download your new excel sheet.